In [29]:
import os

import pandas as pd
import random
from collections import Counter
import warnings
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import spacy
from spacy.util import compounding
from spacy.util import minibatch
import torch
import torch.nn as nn

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

In [30]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

In [31]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
smaple = pd.read_csv('sample_submission.csv')

In [33]:
print(train.shape)
train.head()

(27481, 4)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [34]:
print(test.shape)
test.head()

(3534, 3)


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [35]:
smaple.head()

,textID,selected_text
0,f87dea47db,NaN
1,96d74cb729,NaN
2,eee518ae67,NaN
3,01082688c6,NaN
4,33987a8ee5,NaN


So We have 27486 tweets in the train set and 3535 tweets in the test set

In [36]:
print(train.info())
print("=================")
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB
None


We have one null Value in the train , as the test field for value is NAN we will just remove it. And no nulls in test set.

In [37]:
train.dropna(inplace=True)

In [15]:
train.describe().T

,count,unique,top,freq
textID,27480,27480,8793b4ff93,1
text,27480,27480,mozert requiem tonight,1
selected_text,27480,22463,good,199
sentiment,27480,3,neutral,11117


# Data Loader